In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
# %tensorflow_version 1.x
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization, LSTM ,Bidirectional, GlobalAveragePooling2D,Conv1D,MaxPooling1D,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools
import torch
from google.colab.patches import cv2_imshow
from skimage.io import imread_collection
from keras.models import Model
import cv2
import matplotlib.image as mpimg
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
folder= '/content/gdrive/My Drive/selected50Image_20class'

In [ ]:
data = []
relDirimg=[]
imagePaths = []
categories=[]   
for dir_, _, files in os.walk(folder):
  for fileName in files:
    relDir = fileName.split('_')[0]
    relFile = os.path.join(dir_, fileName)

    if relDir=='n13054560':categories.append(0);imagePaths.append(relFile)
    if relDir=='n11939491':categories.append(1);imagePaths.append(relFile)
    if relDir=='n07873807':categories.append(2);imagePaths.append(relFile)
    if relDir=='n07753592':categories.append(3);imagePaths.append(relFile)
    if relDir=='n04120489':categories.append(4);imagePaths.append(relFile)
    if relDir=='n04086273':categories.append(5);imagePaths.append(relFile)
    if relDir=='n04044716':categories.append(6);imagePaths.append(relFile)
    if relDir=='n03982430':categories.append(7);imagePaths.append(relFile)
    if relDir=='n03888257':categories.append(8);imagePaths.append(relFile)
    if relDir=='n03877472':categories.append(9);imagePaths.append(relFile)
    if relDir=='n03792972':categories.append(10);imagePaths.append(relFile)
    if relDir=='n03792782':categories.append(11);imagePaths.append(relFile)
    if relDir=='n03775071':categories.append(12);imagePaths.append(relFile)
    if relDir=='n03773504':categories.append(13);imagePaths.append(relFile)
    if relDir=='n03709823':categories.append(14);imagePaths.append(relFile)
    if relDir=='n03590841':categories.append(15);imagePaths.append(relFile)
    if relDir=='n03584829':categories.append(16);imagePaths.append(relFile)
    if relDir=='n03452741':categories.append(17);imagePaths.append(relFile)
    if relDir=='n03445777':categories.append(18);imagePaths.append(relFile)
    if relDir=='n03376595':categories.append(19);imagePaths.append(relFile)  


In [ ]:
from pathlib import Path
from PIL import Image
for imagePath in imagePaths:
    im = Image.open(imagePath).convert('RGB')
    data.append(img_to_array(im))

In [ ]:
dataimg=[]
for im in data:
  dataimg.append(cv2.resize(im,(224,224)))

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(dataimg,categories,random_state=100)
x_val,x_test,y_val,y_test = train_test_split(x_val,y_val,random_state=100)

In [ ]:
x_trainn=np.stack( x_train, axis=0 )  
x_vall=np.stack( x_val, axis=0 )  
x_testt=np.stack( x_test, axis=0 )  

In [ ]:
labbtr = to_categorical(y_train)
labbv = to_categorical(y_val)
labbte = to_categorical(y_test)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = base_model.output
x=Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
predictions = Dense(20, activation='softmax')(x)

from tensorflow.keras.models import Model
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.summary() 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
METRICS = [
      keras.metrics.CategoricalAccuracy(name="accuracy"),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn')
]

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

In [ ]:
history=model.fit(x_trainn, labbtr, batch_size=16, epochs=50,validation_data=(x_vall,labbv))

Epoch 1/50
47/47 [==============================] - 18s 162ms/step - loss: 51.5937 - accuracy: 0.4699 - tp: 327.0000 - fp: 305.0000 - tn: 13888.0000 - fn: 420.0000 - val_loss: 3.3051 - val_accuracy: 0.6524 - val_tp: 121.0000 - val_fp: 55.0000 - val_tn: 3498.0000 - val_fn: 66.0000
Epoch 2/50
47/47 [==============================] - 6s 122ms/step - loss: 23.2769 - accuracy: 0.7564 - tp: 565.0000 - fp: 180.0000 - tn: 14013.0000 - fn: 182.0000 - val_loss: 10.8670 - val_accuracy: 0.6310 - val_tp: 118.0000 - val_fp: 68.0000 - val_tn: 3485.0000 - val_fn: 69.0000
Epoch 3/50
47/47 [==============================] - 6s 122ms/step - loss: 5.2604 - accuracy: 0.8447 - tp: 631.0000 - fp: 115.0000 - tn: 14078.0000 - fn: 116.0000 - val_loss: 18.3197 - val_accuracy: 0.6043 - val_tp: 113.0000 - val_fp: 74.0000 - val_tn: 3479.0000 - val_fn: 74.0000
Epoch 4/50
47/47 [==============================] - 6s 122ms/step - loss: 3.0208 - accuracy: 0.8942 - tp: 668.0000 - fp: 79.0000 - tn: 14114.0000 - fn: 79.000

In [ ]:
model.evaluate(x_testt,labbte)

2/2 [==============================] - 6s 3s/step - loss: 106.9976 - accuracy: 0.7619 - tp: 48.0000 - fp: 15.0000 - tn: 1182.0000 - fn: 15.0000


[106.9975814819336, 0.761904776096344, 48.0, 15.0, 1182.0, 15.0]

In [ ]:
from tensorflow.keras.applications import ResNet152V2
pre_trained_model = ResNet152V2(include_top = False,
                             pooling = 'avg',
                            input_shape = (224,224,3),
                            weights = 'imagenet')

# freeze all layers in the base model
pre_trained_model.trainable = False

# un-freeze the BatchNorm layers
for layer in pre_trained_model.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True
last_output = pre_trained_model.output

#Adding some layers to implement ResNet Model to fit well on our Dataset

x = tensorflow.keras.layers.Flatten()(last_output)
x = tensorflow.keras.layers.Dense(2048, activation='relu')(x)
x = tensorflow.keras.layers.Dropout(0.5)(x)
x = tensorflow.keras.layers.Dense(20, activation = 'softmax')(x)
#Model formation

modelr152v2 = tensorflow.keras.Model(pre_trained_model.input, x)
modelr152v2.summary()


234561536/234545216 [==============================] - 3s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                         

In [ ]:
modelr152v2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

In [ ]:
historyr152v2=modelr152v2.fit(x_trainn, labbtr, batch_size=16, epochs=50,validation_data=(x_vall,labbv))

Epoch 1/50
47/47 [==============================] - 34s 388ms/step - loss: 1.2984 - accuracy: 0.6877 - tp: 501.0000 - fp: 106.0000 - tn: 15284.0000 - fn: 309.0000 - val_loss: 1.4984 - val_accuracy: 0.7219 - val_tp: 129.0000 - val_fp: 35.0000 - val_tn: 3518.0000 - val_fn: 58.0000
Epoch 2/50
47/47 [==============================] - 14s 299ms/step - loss: 0.2871 - accuracy: 0.9143 - tp: 665.0000 - fp: 37.0000 - tn: 14156.0000 - fn: 82.0000 - val_loss: 0.5738 - val_accuracy: 0.8663 - val_tp: 160.0000 - val_fp: 22.0000 - val_tn: 3531.0000 - val_fn: 27.0000
Epoch 3/50
47/47 [==============================] - 14s 306ms/step - loss: 0.1312 - accuracy: 0.9558 - tp: 702.0000 - fp: 24.0000 - tn: 14169.0000 - fn: 45.0000 - val_loss: 0.4691 - val_accuracy: 0.8930 - val_tp: 167.0000 - val_fp: 18.0000 - val_tn: 3535.0000 - val_fn: 20.0000
Epoch 4/50
47/47 [==============================] - 14s 307ms/step - loss: 0.1241 - accuracy: 0.9652 - tp: 715.0000 - fp: 20.0000 - tn: 14173.0000 - fn: 32.0000 - v

In [ ]:
modelr152v2.evaluate(x_testt,labbte)

2/2 [==============================] - 2s 1s/step - loss: 0.4866 - accuracy: 0.9206 - tp: 58.0000 - fp: 4.0000 - tn: 1193.0000 - fn: 5.0000


[0.4866023659706116, 0.920634925365448, 58.0, 4.0, 1193.0, 5.0]

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2

pre_trained_model = InceptionResNetV2(weights='imagenet', pooling = 'avg', include_top=False, input_shape=[224, 224, 3])   

# freeze all layers in the base model
pre_trained_model.trainable = False

for layer in pre_trained_model.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True
last_output = pre_trained_model.output

#Adding some layers to implement ResNet Model to fit well on our Dataset

x = tensorflow.keras.layers.Flatten()(last_output)
x = tensorflow.keras.layers.Dense(2048, activation='relu')(x)
x = tensorflow.keras.layers.Dropout(0.5)(x)
x = tensorflow.keras.layers.Dense(20, activation = 'softmax')(x)
#Model formation

modelir = tensorflow.keras.Model(pre_trained_model.input, x)

modelir.summary()

219070464/219055592 [==============================] - 4s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)              

In [ ]:
modelir.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

In [ ]:
historyrir=modelir.fit(x_trainn, labbtr, batch_size=16, epochs=50,validation_data=(x_vall,labbv))

Epoch 1/50
47/47 [==============================] - 36s 354ms/step - loss: 1.1093 - accuracy: 0.7370 - tp: 534.0000 - fp: 49.0000 - tn: 15341.0000 - fn: 276.0000 - val_loss: 0.5444 - val_accuracy: 0.9091 - val_tp: 169.0000 - val_fp: 14.0000 - val_tn: 3539.0000 - val_fn: 18.0000
Epoch 2/50
47/47 [==============================] - 11s 233ms/step - loss: 0.3045 - accuracy: 0.9076 - tp: 658.0000 - fp: 31.0000 - tn: 14162.0000 - fn: 89.0000 - val_loss: 0.3254 - val_accuracy: 0.9251 - val_tp: 172.0000 - val_fp: 7.0000 - val_tn: 3546.0000 - val_fn: 15.0000
Epoch 3/50
47/47 [==============================] - 11s 234ms/step - loss: 0.1911 - accuracy: 0.9438 - tp: 692.0000 - fp: 24.0000 - tn: 14169.0000 - fn: 55.0000 - val_loss: 0.3740 - val_accuracy: 0.9251 - val_tp: 172.0000 - val_fp: 11.0000 - val_tn: 3542.0000 - val_fn: 15.0000
Epoch 4/50
47/47 [==============================] - 11s 239ms/step - loss: 0.1635 - accuracy: 0.9545 - tp: 700.0000 - fp: 20.0000 - tn: 14173.0000 - fn: 47.0000 - val

In [ ]:
modelir.evaluate(x_testt,labbte)

2/2 [==============================] - 3s 1s/step - loss: 0.3106 - accuracy: 0.9683 - tp: 60.0000 - fp: 2.0000 - tn: 1195.0000 - fn: 3.0000


[0.3105796277523041, 0.9682539701461792, 60.0, 2.0, 1195.0, 3.0]